# CS50P - Semana 6: Entrada y Salida de Archivos (File I/O)

---

Hasta ahora, todos nuestros datos vivían en la memoria RAM; al cerrar el programa, la información desaparecía. En esta unidad aprenderemos a persistir datos usando archivos de texto y formatos estructurados como CSV.

### Objetivos de Aprendizaje
* **Escribir** datos en archivos externos para guardarlos permanentemente.
* **Leer** información desde archivos para procesarla en Python.
* **Manipular** archivos CSV (Comma-Separated Values) usando la librería estándar.
* **Ordenar** datos complejos usando funciones lambda.

---

### Índice de Contenidos
1. [Escritura de Archivos: open y "w"](#writing)
2. [El contexto 'with' (The Pythonic Way)](#with)
3. [Lectura de Archivos: open y "r"](#reading)
4. [Archivos CSV (Comma-Separated Values)](#csv)
5. [Ordenación con Lambdas](#sorting)
6. [La Librería Estándar csv (DictReader)](#csv-library)
7. [Escribir Datos: DictWriter](#csv-writing)
8. [Archivos Binarios y PIL (Pillow)](#binary-pil)
9. [Resumen de la Semana 6](#resumen-6)

<h2 id="writing">1. Escritura de Archivos: open y "w"</h2>

---

Para interactuar con un archivo, usamos la función integrada `open()`. Esta función requiere dos argumentos principales: el nombre del archivo y el **modo**.

* **"w" (write)**: Crea el archivo o sobrescribe su contenido.
* **"a" (append)**: Añade contenido al final del archivo sin borrar lo anterior.



### Ejemplo Inicial: Guardar nombres

In [ ]:
# names.py
name = input("What's your name? ")

# Abrimos el archivo en modo "a" (append) para no borrar los nombres anteriores
file = open("names.txt", "a")
file.write(f"{name}\n") # Importante añadir \n para el salto de línea
file.close() # Es vital cerrar el archivo manualmente en este método

<h2 id="with">2. El Contexto 'with' (The Pythonic Way)</h2>

---

Olvidar cerrar un archivo con `.close()` puede corromper los datos. Python ofrece una forma más segura y elegante de manejar archivos usando la palabra clave **`with`**.

Con `with`, Python cierra el archivo automáticamente en cuanto termina el bloque de código, incluso si ocurre un error.

In [ ]:
# names.py mejorado
name = input("What's your name? ")

# No necesitamos file.close(), Python lo hace por nosotros
with open("names.txt", "a") as file:
    file.write(f"{name}\n")

<h2 id="reading">3. Lectura de Archivos: open y "r"</h2>

---

Para leer información, usamos el modo **"r" (read)**. Podemos iterar sobre el archivo línea por línea como si fuera una lista.

### Métodos comunes:
* **`.readlines()`**: Lee todo el archivo y lo mete en una lista.
* **Iteración directa**: Más eficiente para archivos grandes ya que lee línea a línea.

In [ ]:
# Leyendo y limpiando los nombres guardados
with open("names.txt", "r") as file:
    for line in file:
        # .rstrip() elimina el salto de línea invisible al final de cada nombre
        print(f"hello, {line.rstrip()}")

### Ordenar la información

Notarás que leer el archivo directamente imprime los nombres en el orden en que fueron guardados. Si queremos presentarlos de forma profesional (alfabéticamente), debemos seguir este flujo:

1.  Crear una lista vacía en memoria.
2.  Leer el archivo y añadir cada elemento a la lista limpiando los espacios (`append` + `rstrip`).
3.  Imprimir la lista utilizando la función **`sorted()`**.

[Image showing data flowing from a text file into a Python list and then being processed by the sorted() function]

In [ ]:
names = []

# 1. Recolectamos los datos
with open("names.txt") as file:
    for line in file:
        names.append(line.rstrip())

# 2. Los presentamos ordenados
for name in sorted(names):
    print(f"hello, {name}")

<h2 id="csv">4. Archivos CSV (Comma-Separated Values)</h2>

---

Los archivos **CSV** ("valores separados por comas") son un estándar sencillo para almacenar datos tabulares en texto plano. Cada línea del archivo representa un registro y cada coma actúa como frontera entre columnas.

### Estructura de nuestro archivo de ejemplo
Para empezar, imagina un archivo llamado `students.csv` con el siguiente contenido:

```text
Hermione,Gryffindor
Harry,Gryffindor
Ron,Gryffindor
Draco,Slytherin
```

### Lectura Manual: El método `split`

Podemos leer un archivo CSV utilizando herramientas básicas de strings. La función clave aquí es **`.split(",")`**, que divide una cadena de texto en una lista cada vez que encuentra una coma.

* **`rstrip()`**: Elimina el salto de línea al final para evitar espacios vacíos.
* **`split(",")`**: Indica al intérprete dónde termina cada valor.
* **Índices**: `row[0]` accede al primer elemento (nombre) y `row[1]` al segundo (casa).

In [ ]:
# students.py - Procesamiento por índices
with open("students.csv") as file:
    for line in file:
        row = line.rstrip().split(",")
        print(f"{row[0]} is in {row[1]}")

### Mejorando la legibilidad: Desempaquetado

Usar índices como `row[0]` puede ser críptico. Python permite asignar múltiples valores a la vez si sabemos exactamente cuántos elementos devolverá el `split`. Esto hace que el código se documente a sí mismo.

In [ ]:
# students.py - Asignación directa
with open("students.csv") as file:
    for line in file:
        name, house = line.rstrip().split(",")
        print(f"{name} is in {house}")

### Organizando Datos: Listas de Diccionarios

Si queremos ordenar la salida, primero debemos recolectar todos los datos en memoria. Aunque podrías usar una lista de strings, es mucho más potente usar **diccionarios**, donde asociamos una "llave" (key) con un valor.

1.  Creamos una lista vacía `students = []`.
2.  Para cada línea, creamos un diccionario temporal `{"name": name, "house": house}`.
3.  Añadimos el diccionario a nuestra lista para procesarlo después.

In [ ]:
students = []

with open("students.csv") as file:
    for line in file:
        name, house = line.rstrip().split(",")
        # Creamos el diccionario y lo añadimos en un solo paso
        students.append({"name": name, "house": house})

# Imprimimos los datos recolectados
for student in students:
    print(f"{student['name']} is in {student['house']}")

<h2 id="sorting">5. Ordenación con Lambdas</h2>

---

A diferencia de una lista simple, no podemos ordenar una lista de diccionarios directamente porque Python no sabe si quieres ordenar por "name" o por "house".

Para solucionar esto, usamos el parámetro `key` de la función `sorted()`:

* **Funciones con nombre**: Podemos definir una función `get_name(student)` que retorne el valor de la llave por la que queremos ordenar.
* **Funciones Lambda**: Si solo usaremos la función una vez, podemos usar una función anónima (lambda) que actúa como un atajo: `lambda student: student["name"]`.

In [ ]:
students = []

with open("students.csv") as file:
    for line in file:
        name, house = line.rstrip().split(",")
        students.append({"name": name, "house": house})

# Ordenamos usando una función lambda para decirle a sorted() que use el nombre
for student in sorted(students, key=lambda student: student["name"]):
    print(f"{student['name']} is in {student['house']}")

### El Problema de la "Coma Extra"

Nuestro programa es frágil. ¿Qué sucede si un valor contiene una coma? Por ejemplo, si Harry vive en `"Number Four, Privet Drive"`:

```text
Harry,"Number Four, Privet Drive"
Ron,The Burrow
```

Al usar `.split(",")`, Python encontrará TRES valores en la línea de Harry, lanzando un error de tipo **`ValueError: too many values to unpack`**. Aquí es donde necesitamos herramientas profesionales.

<h2 id="csv-library">6. La Librería Estándar `csv`</h2>

---

Python incluye una librería llamada `csv` diseñada específicamente para manejar estas complejidades.

### csv.reader
El objeto `reader` analiza el archivo y sabe manejar comas dentro de comillas automáticamente. Nos devuelve cada fila como una lista de valores.

### csv.DictReader
Es la forma más flexible y robusta (defensiva) de programar. Si nuestro archivo CSV tiene una **cabecera** (headers), `DictReader` convierte cada fila automáticamente en un diccionario usando los nombres de las columnas como llaves.

In [ ]:
import csv

students = []

# Supongamos que el archivo empieza con: name,house
with open("students.csv") as file:
    # DictReader asume que la primera línea son las llaves
    reader = csv.DictReader(file)
    for row in reader:
        # 'row' ya es un diccionario con las llaves 'name' y 'house'
        students.append({"name": row["name"], "house": row["house"]})

for student in sorted(students, key=lambda student: student["name"]):
    print(f"{student['name']} is from {student['house']}")

<h2 id="csv-writing">7. Escribir Datos: DictWriter</h2>

---

Para guardar información, usamos **`csv.DictWriter`**. Este requiere dos parámetros: el archivo donde escribir y una lista de los nombres de los campos (`fieldnames`).

* **`writerow()`**: Recibe un diccionario y escribe los valores en las columnas correspondientes de forma segura.

In [ ]:
import csv

name = input("What's your name? ")
house = input("Where's your house? ") # Cambiado de house a house por consistencia

# 1. Añadimos newline='' para evitar filas vacías fantasma
with open("students.csv", "a", newline='') as file:
    # 2. El fieldnames debe coincidir exactamente con el DictReader anterior
    writer = csv.DictWriter(file, fieldnames=["name", "house"])
    
    # 3. Nota: Solo escribiríamos writer.writeheader() si el archivo estuviera vacío
    writer.writerow({"name": name, "house": house})

<h2 id="binary-pil">8. Archivos Binarios y PIL (Pillow)</h2>

---

A diferencia de los archivos de texto que hemos visto (como `.txt` o `.csv`), un **archivo binario** es simplemente una colección de unos y ceros (bits). Este formato se utiliza para almacenar datos complejos como música, videos o imágenes.

Para trabajar con imágenes en Python, utilizamos una librería muy popular llamada **PIL** (Python Imaging Library), ahora conocida como **Pillow**.



### GIFs Animados
Un GIF animado es un tipo de archivo de imagen que contiene múltiples imágenes internas. Estas se reproducen en secuencia una y otra vez para crear un efecto de video o animación sencilla.

### Creación de una Animación: `costumes.py`

Imagina que tienes dos imágenes, `costume1.gif` y `costume2.gif`, donde la posición de las piernas es ligeramente diferente. Podemos usar Python para combinarlas en un bucle infinito.

**Conceptos clave del código:**
* **`sys.argv[1:]`**: Usamos el "slicing" de listas para tomar todos los argumentos de la línea de comandos excepto el nombre del script.
* **`Image.open(arg)`**: Abre cada archivo de imagen proporcionado.
* **`.save()`**: Este método tiene parámetros especiales para animaciones:
    * `save_all=True`: Asegura que se guarden todos los marcos (frames).
    * `append_images`: Recibe una lista con el resto de las imágenes a añadir.
    * `duration`: Tiempo en milisegundos entre cada marco.
    * `loop=0`: Define que la animación se repita infinitamente.

###  Recuerda instalar la librería antes de correr esta sección:
pip install Pillow

In [ ]:
# costumes.py
import sys
from PIL import Image

images = []

# Iteramos sobre los nombres de archivos pasados por terminal
for arg in sys.argv[1:]:
    image = Image.open(arg)
    images.append(image)

# Guardamos la primera imagen y añadimos las demás para crear el GIF
images[0].save(
    "costumes.gif", 
    save_all=True, 
    append_images=images[1:], # Añadimos el resto de la lista
    duration=200,             # 200 milisegundos por cuadro
    loop=0                    # Bucle infinito
)

### Cómo ejecutar el programa
Para generar la animación, debes tener las imágenes en la misma carpeta y ejecutar en tu terminal:

```bash
python costumes.py costume1.gif costume2.gif
```

Esto generará un nuevo archivo llamado `costumes.gif` que podrás abrir en tu navegador o visualizador de imágenes para ver la animación.

<h2 id="resumen-6">9. Resumen de la Semana 6: La Memoria Permanente</h2>

---

Esta semana marca el paso de programas efímeros a aplicaciones capaces de gestionar información a largo plazo. Aquí tienes los pilares fundamentales y su documentación de referencia:

* **Persistencia (Lectura y Escritura)**: Aprendimos a usar la función integrada [`open()`](https://docs.python.org/3/library/functions.html#open) para mover datos entre la memoria RAM (volátil) y el disco duro (permanente).
* **Seguridad y Contexto**: Implementamos la sentencia [`with`](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) (Context Managers), que automatiza el cierre de flujos de datos para prevenir fugas de memoria o archivos corrompidos.
* **Estructura de Datos (CSV)**: Utilizamos el módulo estándar [`csv`](https://docs.python.org/3/library/csv.html), destacando herramientas como [`DictReader`](https://docs.python.org/3/library/csv.html#csv.DictReader) y [`DictWriter`](https://docs.python.org/3/library/csv.html#csv.DictWriter) para manejar tablas de datos de forma profesional y defensiva.
* **Archivos Binarios e Imágenes**: Exploramos la manipulación de flujos binarios mediante la librería externa [`Pillow` (PIL)](https://pillow.readthedocs.io/en/stable/), permitiendo la creación de archivos complejos como GIFs animados.





---

### 📚 Bibliografía Oficial Consultada
* [Python Documentation: Input and Output](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files)
* [Python Documentation: The `csv` File Reading and Writing](https://docs.python.org/3/library/csv.html)
* [Pillow (PIL) Handbook](https://pillow.readthedocs.io/en/stable/handbook/index.html)

---
